In [ ]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp as OR

In [ ]:
graph = pd.read_csv('mis_seat_packing_example.csv',index_col=0).astype(int)
graph.columns = graph.columns.astype(int)

In [ ]:
def mis(graph, integer=False):
    """A model for solving the maximum independent set problem.
    
    Args:
        graph (pd.DataFrame): Graph represented by an adjacency matrix.
    """
    NODES = list(graph.index)                # nodes
    EDGES = []                               # edges
    for i in NODES:
        for j in NODES:
            if i <= j and graph.at[i,j] == 1:
                EDGES.append((i,j))
    
    # define model
    m = OR.Solver('mis', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    x = {} 
    for i in NODES:
        if integer:
            x[i] = m.IntVar(0, 1, ('%s' % (i)))
        else:
            x[i] = m.NumVar(0, 1, ('%s' % (i)))
    
    # objective function
    m.Maximize(sum(x[i] for i in NODES))
        
    # subject to: no vertices in the set share an edge
    for i,j in EDGES:
        m.Add(x[i] + x[j] <= 1)

    return m,x

In [ ]:
def solve(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())
    for var in m.variables():
        print(var.name(), ':',  var.solution_value())

In [ ]:
m,x = mis(graph)
solve(m)

In [ ]:
m,x = mis(graph, integer=True)
solve(m)